In [0]:
import plotly.offline as py
import plotly.graph_objs as go
import pandas as pd

py.offline.init_notebook_mode(connected=True)

#import data

#datapath = "C:\Users\Dan Steffy\Dropbox\Research\VRPVIS\\"
datapath = "/Users/TomCreo/Desktop/"

df_ArrivalTimes = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="ArrivalTimes")
df_Locations = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="Locations")
df_ArrivalLoad = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="ArrivalLoad")
df_Demand = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="Demand")
df_Requests = pd.read_csv(str(datapath+"Google_Campus_Data_217_Shifted.csv"))

#to store requestID associated with each stop
VehRequestID = []

#to store latitude associated with each stop
VehLat = []

#to store longitudes associated with each stop
VehLong = []

#marker fill based on pickup or dropoff
VehFill = []

#hovertext RequestID
VehText = []


#create lists of lists
for v in range(30):
    VehRequestID.append([])
    VehLat.append([])
    VehLong.append([])
    VehFill.append([])
    VehText.append([])
    
#calculate request ID associated with each stop
#extract latitude/longitude from requestID
#assign marker fill: pickup = 'circle', dropoff = 'circle-open'

clist = df_Locations.columns
for v in range(30):
    for i, location in enumerate(df_Locations[clist[v]].dropna(),start=1):
        vehdemand =  df_Demand[clist[v]].dropna()[i]
        if vehdemand == 1:
            VehRequestID[v].append(location/2)
            VehLat[v].append(df_Requests.fromLatitude[location/2])
            VehLong[v].append(df_Requests.fromLongitude[location/2])
            VehFill[v].append('circle')
            VehText[v].append('Pickup RequestID '+str(int(location/2)))
        elif vehdemand == -1:
            VehRequestID[v].append((location-1)/2)
            VehLat[v].append(df_Requests.toLatitude[(location-1)/2])
            VehLong[v].append(df_Requests.toLongitude[(location-1)/2])
            VehFill[v].append('circle-open')
            VehText[v].append('Dropoff RequestID '+str(int((location-1)/2)))

            
#define minute by minute interval between first arrival time and last in solution data
interval = range(27010,37388,60)

#create list of positions associated with interval for vehicle
VehPosition = []
            
#minute by minute position for Veh0 all stops
for i in range(len(VehLong[0])-1):
    for z in interval:
        if df_ArrivalTimes[clist[0]].dropna()[i+1] <= z <= df_ArrivalTimes[clist[0]].dropna()[i+2]:
            t = (z - df_ArrivalTimes[clist[0]].dropna()[i+1]) / (df_ArrivalTimes[clist[0]].dropna()[i+2]-df_ArrivalTimes[clist[0]].dropna()[i+1])
            x = VehLong[0][i] + (VehLong[0][i+1]-VehLong[0][i])*t
            y = VehLat[0][i] + (VehLat[0][i+1]-VehLat[0][i])*t
            VehPosition.append((x, y, z))
            
#extract x, y, z coords from positions
positionx = [i[0] for i in VehPosition]
positiony = [i[1] for i in VehPosition]
positionz = [i[2] for i in VehPosition]
    

data = []
for v in range(30):
    trace = go.Scatter3d(
        x=VehLong[v],
        y=VehLat[v],
        z=df_ArrivalTimes[clist[v]].dropna(),
        name='Vehicle' + str(v),
        mode='lines+markers',
        marker=dict(
            size=6,
            symbol=VehFill[v],
            line=dict(
                width=0.5
            ),
            opacity=0.7
        ),
        line=dict(),
        text=VehText[v]
    )
    data.append(trace)

    
#trace to check positions against veh0 route
traceposition = go.Scatter3d(
        x=positionx,
        y=positiony,
        z=positionz,
        name='Veh0 positions',
        mode='markers',
        marker=dict(
            size=3,
            symbol='square',
            line=dict(
                width=0.5
            ),
            opacity=0.7
        ),
        line=dict(),
    )
data.append(traceposition)
    
    
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    ),
    legend=dict(
        x=0,
        y=0,
        traceorder='normal',
        font=dict(
            family='sans-serif',
            size=12,
            color='#000'
        ),
        bgcolor='#E2E2E2',
        bordercolor='#FFFFFF',
        borderwidth=2
    )
)

fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig)



In [48]:
import plotly.offline as py
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import itertools
from haversine import haversine
from scipy import stats


py.offline.init_notebook_mode(connected=True)

#import data

#datapath = "C:\Users\Dan Steffy\Dropbox\Research\VRPVIS\\"
datapath = "/Users/TomCreo/Desktop/"

df_ArrivalTimes = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="ArrivalTimes")
df_Locations = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="Locations")
df_ArrivalLoad = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="ArrivalLoad")
df_Demand = pd.read_excel(str(datapath+"Assignment30.xlsx"), sheet_name="Demand")
df_Requests = pd.read_csv(str(datapath+"Google_Campus_Data_217_Shifted.csv"))

#to store requestID associated with each stop
VehRequestID = []

#to store latitude associated with each stop
VehLat = []

#to store longitudes associated with each stop
VehLong = []

#marker fill based on pickup or dropoff
VehFill = []

#hovertext RequestID
VehText = []


#create lists of lists
for v in range(30):
    VehRequestID.append([])
    VehLat.append([])
    VehLong.append([])
    VehFill.append([])
    VehText.append([])
    
#calculate request ID associated with each stop
#extract latitude/longitude from requestID
#assign marker fill: pickup = 'circle', dropoff = 'circle-open'

clist = df_Locations.columns
for v in range(30):
    for i, location in enumerate(df_Locations[clist[v]].dropna(),start=1):
        vehdemand =  df_Demand[clist[v]].dropna()[i]
        if vehdemand == 1:
            VehRequestID[v].append(location/2)
            VehLat[v].append(df_Requests.fromLatitude[location/2])
            VehLong[v].append(df_Requests.fromLongitude[location/2])
            VehFill[v].append('circle')
            VehText[v].append('Pickup RequestID '+str(int(location/2)))
        elif vehdemand == -1:
            VehRequestID[v].append((location-1)/2)
            VehLat[v].append(df_Requests.toLatitude[(location-1)/2])
            VehLong[v].append(df_Requests.toLongitude[(location-1)/2])
            VehFill[v].append('circle-open')
            VehText[v].append('Dropoff RequestID '+str(int((location-1)/2)))

            
            
#define minute by minute interval between first arrival time and last in solution data
interval = range(27010,37388,60)

            
#minute by minute position for given vehicle between all stops
def position(VehNum):
    VehPosition = []
    for i in range(len(VehLong[VehNum])-1):
        for z in interval:
            if df_ArrivalTimes[clist[VehNum]].dropna()[i+1] <= z <= df_ArrivalTimes[clist[VehNum]].dropna()[i+2]:
                t = ((z - df_ArrivalTimes[clist[VehNum]].dropna()[i+1]) / 
                        (df_ArrivalTimes[clist[VehNum]].dropna()[i+2]-df_ArrivalTimes[clist[VehNum]].dropna()[i+1])
                    )
                x = VehLong[VehNum][i] + (VehLong[VehNum][i+1]-VehLong[VehNum][i])*t
                y = VehLat[VehNum][i] + (VehLat[VehNum][i+1]-VehLat[VehNum][i])*t
                VehPosition.append((x, y, z))
    return VehPosition;



#calculates min by min position and stores in data frame

#VehiclePositions = pd.DataFrame()
#for i in range(30):
#  VehiclePositions = VehiclePositions.append(position(i))



#calculates avg location for 2 specified vehicles and measures distance between them
#returns distance between avg locations in km

def avg_loc_dist(VehNum1,VehNum2):
    AvgLong1 = sum(VehLong[VehNum1]) / len(VehLong[VehNum1])
    AvgLat1 = sum(VehLat[VehNum1]) / len(VehLat[VehNum1])
    AvgLoc1 = (AvgLong1, AvgLat1)

    AvgLong2 = sum(VehLong[VehNum2]) / len(VehLong[VehNum2])
    AvgLat2 = sum(VehLat[VehNum2]) / len(VehLat[VehNum2])
    AvgLoc2 = (AvgLong2, AvgLat2)
    
    dist = haversine(AvgLoc1, AvgLoc2)
    return dist;




#calculate distance in km for all combinations of 2 vehicles and find minimum overall vehicles
AvgLocDist = []
AvgLocMinDist = None

for pair in itertools.combinations(range(30), 2):
    CurrentDist = avg_loc_dist(*pair)
    AvgLocDist.append([pair, CurrentDist])
    print [pair, CurrentDist]
    if AvgLocMinDist == None:
        AvgLocMinDist = CurrentDist
    else:
        AvgLocMinDist = min(AvgLocMinDist,CurrentDist)


        
    
    
#calculates min distance between 2 specified vehicles and finds time with min distance 
#returns minimum distance
def min_dist(VehNum1,VehNum2):
    dist2 = []
    VehNum1Pos = position(VehNum1)
    VehNum2Pos = position(VehNum2)
    for t in interval:
        VehNum1Loc_t = [item for item in VehNum1Pos if item[2] == t]
        VehNum2Loc_t = [item for item in VehNum2Pos if item[2] == t]
        if (len(VehNum1Loc_t) != 0 and
            len(VehNum2Loc_t) != 0):
            VehNum1Loc = (VehNum1Loc_t[0][0],VehNum1Loc_t[0][1])
            VehNum2Loc = (VehNum2Loc_t[0][0],VehNum2Loc_t[0][1])
            dist2.append(((haversine(VehNum1Loc, VehNum2Loc))))
    return min(dist2);



#caclulates min distance for all pairs of vehicles
MinLocDist = []

#for pair in itertools.combinations(range(30), 2):
#    min_distance = min_dist(*pair)
#    MinLocDist.append([pair, min_distance])
    
#print MinLocDist

    
    
#calculates avg distance between 2 specified vehicles over time when both are active; returns distance
def avg_dist(VehNum1,VehNum2):
    Distance_t = min_dist(VehNum1,VehNum2)
    DistSum = sum(pair[0] for pair in Distance_t)
    AvgDist = DistSum / len(Distance_t)
    return AvgDist;





#read in experiment output and calculate improvements
df_ExpData = pd.read_excel(str(datapath+"Data 435.xlsx"))

ReoptImprovement = []

for i in range(len(df_ExpData.FirstVehicle)-1):
    ReoptImprovement.append([(int(df_ExpData.FirstVehicle[i+1]), int(df_ExpData.SecondVehicle[i+1])), (int(df_ExpData.OriginalCost_s[i+1]) - int(df_ExpData.MIPTrueCost_s[i+1]))])
    
    



#create 2d scatterplots

ReoptImpY = [i[1] for i in ReoptImprovement]
VehPairs = [i[0] for i in ReoptImprovement]


#versus AvgDist
AvgDistX = [i[1] for i in AvgLocDist]

slope1, intercept1, r_value, p_value, std_err = stats.linregress(AvgDistX,ReoptImpY)

line1 = slope1*np.array(AvgDistX)+intercept1



trace0 = go.Scatter(
    x = AvgDistX,
    y = ReoptImpY,
    mode = 'markers',
    text = VehPairs,
    name = 'Vehicle Route Pairs'
    )

trace1 = go.Scatter(
    x=AvgDistX,
    y=line1,
    mode='lines',
    name='Linear Fit'
    )

data1 = [trace0,trace1]

layout1 = go.Layout(
    title = 'Distance Between Average Locations of Route Pairs vs. Reoptimized Improvement',
    font=dict(family='Arial, sans-serif', size=12),
    xaxis = dict(
        title = 'Distance Between Average Locations of Route Pairs (km)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=12
        ),
        tickfont=dict(
            family='Arial, sans-serif',
            size=10
        ),
    ),
    yaxis = dict(
        title = 'Route Cost Improvement upon Reoptimizing Route Pair (s)',
        titlefont=dict(
            family='Arial, sans-serif',
            size=12
        ),
        tickfont=dict(
            family='Arial, sans-serif',
            size=10
        )
    )
)

fig1 = dict(data=data1, layout=layout1)

py.offline.iplot(fig1)

print "R value: "
print r_value

[(0, 1), 3.574332187435159]
[(0, 2), 0.5578911205632264]
[(0, 3), 0.40774363896304755]
[(0, 4), 4.066508149500976]
[(0, 5), 3.149885962029758]
[(0, 6), 1.594916372047605]
[(0, 7), 0.7941387493860927]
[(0, 8), 3.1286652429859276]
[(0, 9), 0.8763114704228396]
[(0, 10), 4.1583836805284236]
[(0, 11), 1.435586627258154]
[(0, 12), 1.7403402373355037]
[(0, 13), 2.5232586201585088]
[(0, 14), 1.7423801573343984]
[(0, 15), 1.160500061934107]
[(0, 16), 1.7209875465184532]
[(0, 17), 1.5228129367176173]
[(0, 18), 5.287519875654564]
[(0, 19), 1.3943072238002037]
[(0, 20), 5.892276494237751]
[(0, 21), 4.685651250056291]
[(0, 22), 2.104190354953844]
[(0, 23), 4.613808053884752]
[(0, 24), 0.46658146750407137]
[(0, 25), 1.399260821883195]
[(0, 26), 2.421614446889693]
[(0, 27), 0.5011716521917313]
[(0, 28), 0.3777839669095746]
[(0, 29), 1.4245213900420817]
[(1, 2), 3.9462282515562674]
[(1, 3), 3.5327411318563557]
[(1, 4), 0.5696218825979942]
[(1, 5), 0.42450682685192787]
[(1, 6), 2.008284403700675]
[(1, 

R value: 
-0.045696157236033694
